# Clustering Crypto

In [116]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from path import Path
import numpy as np
%matplotlib inline

### Fetching Cryptocurrency Data

In [117]:
# Alternatively, use the provided csv file:
file_path = Path("Resources/crypto_data.csv")

# Create a DataFrame
crypto_df = pd.read_csv(file_path)
crypto_df


,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...,...
1247,XBC,BitcoinPlus,Scrypt,True,PoS,1.283270e+05,1000000
1248,DVTC,DivotyCoin,Scrypt,False,PoW/PoS,2.149121e+07,100000000
1249,GIOT,Giotto Coin,Scrypt,False,PoW/PoS,NaN,233100000
1250,OPSC,OpenSourceCoin,SHA-256,False,PoW/PoS,NaN,21000000


### Data Preprocessing

In [118]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
crypto_df = crypto_df.set_index("Unnamed: 0")


In [119]:
crypto_df.index.name = None

In [120]:
# Keep only cryptocurrencies that are trading

crypto_df = crypto_df.loc[crypto_df['IsTrading'] == True]
crypto_df

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
SERO,Super Zero,Ethash,True,PoW,NaN,1000000000
UOS,UOS,SHA-256,True,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,True,PoW,7.296538e+06,21000000


In [121]:
# Keep only cryptocurrencies with a working algorithm

crypto_df = crypto_df.loc[crypto_df['Algorithm'] != 'N/A']
crypto_df

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
SERO,Super Zero,Ethash,True,PoW,NaN,1000000000
UOS,UOS,SHA-256,True,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,True,PoW,7.296538e+06,21000000


In [122]:
# Remove the "IsTrading" column

crypto_df = crypto_df.drop(columns = 'IsTrading')
crypto_df


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...
SERO,Super Zero,Ethash,PoW,NaN,1000000000
UOS,UOS,SHA-256,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [123]:
# Remove rows with at least 1 null value
crypto_df = crypto_df.dropna()

In [124]:
# Remove rows with cryptocurrencies having no coins mined

crypto_df = crypto_df.loc[crypto_df['TotalCoinsMined'] != 0]
crypto_df


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [125]:
# Drop rows where there are 'N/A' text values

crypto_df = crypto_df[crypto_df!='N/A']
crypto_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [126]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
coins_df = pd.DataFrame(crypto_df["CoinName"], index= crypto_df.index)
coins_df


,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum
...,...
ZEPH,ZEPHYR
GAP,Gapcoin
BDX,Beldex
ZEN,Horizen


In [127]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_df = crypto_df.drop(columns = 'CoinName')
crypto_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [128]:
# Create dummy variables for text features
crypto_dummy= pd.get_dummies(crypto_df, columns=['Algorithm', 'ProofType'])
crypto_dummy.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [129]:
# Standardize data

crypto_scaled = StandardScaler().fit_transform(crypto_dummy)
crypto_scaled

array([[-0.11674788, -0.15286468, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       [-0.09358885, -0.14499604, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       [ 0.52587231,  4.4937636 , -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       ...,
       [-0.09523411, -0.13215444, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       [-0.11658774, -0.15255408, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       [-0.11674507, -0.15284989, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ]])

### Reducing Dimensions Using PCA

In [130]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)

cryptocurrencies_pca = pca.fit_transform(crypto_scaled)

In [131]:
# Create a DataFrame with the principal components data
pca_df = pd.DataFrame(cryptocurrencies_pca, columns=["PC1", "PC2", "PC3"],index= crypto_df.index)

pca_df

,PC1,PC2,PC3
42,-0.327718,1.020263,-0.582540
404,-0.311042,1.020787,-0.583118
1337,2.294060,1.628318,-0.664171
BTC,-0.146548,-1.357352,0.235012
ETH,-0.157070,-1.984168,0.415349
...,...,...,...
ZEPH,2.465179,0.457287,0.266171
GAP,-0.325760,1.020199,-0.582576
BDX,0.326367,-2.276161,0.455099
ZEN,-0.177756,-1.938128,0.338962


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [88]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(cryptocurrencies_pca)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot
elbow_curve = {"k": k, "inertia": inertia}
elbow_df = pd.DataFrame(elbow_curve)
elbow_df.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

:Curve   [k]   (inertia)

Running K-Means with `k=<your best value for k here>`

In [89]:
# Initialize the K-Means model
model = KMeans(n_clusters=5, random_state=1)

# Fit the model
model.fit(pca_df)

# Predict clusters
predict = model.predict(pca_df)

# Create a new DataFrame including predicted clusters and cryptocurrencies features
pca_df["class"] = model.labels_
pca_df


,PC1,PC2,PC3,class
Unnamed: 0,,,,
42,-0.339516,0.989275,-0.572531,0
404,-0.322849,0.989395,-0.573035,0
1337,2.300148,1.549771,-0.609963,4
BTC,-0.137611,-1.303372,0.206958,1
ETH,-0.142663,-1.990994,0.403226,1
...,...,...,...,...
ZEPH,2.485288,0.843631,-0.111688,4
GAP,-0.337561,0.989155,-0.572553,0
BDX,0.327547,-2.258238,0.463220,1


In [90]:
clustered_df = pd.concat([crypto_df,coins_df, pca_df], axis=1, join="inner")

clustered_df

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,CoinName,PC1,PC2,PC3,class
Unnamed: 0,,,,,,,,,
42,Scrypt,PoW/PoS,4.199995e+01,42,42 Coin,-0.339516,0.989275,-0.572531,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,404Coin,-0.322849,0.989395,-0.573035,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,EliteCoin,2.300148,1.549771,-0.609963,4
BTC,SHA-256,PoW,1.792718e+07,21000000,Bitcoin,-0.137611,-1.303372,0.206958,1
ETH,Ethash,PoW,1.076842e+08,0,Ethereum,-0.142663,-1.990994,0.403226,1
...,...,...,...,...,...,...,...,...,...
ZEPH,SHA-256,DPoS,2.000000e+09,2000000000,ZEPHYR,2.485288,0.843631,-0.111688,4
GAP,Scrypt,PoW/PoS,1.493105e+07,250000000,Gapcoin,-0.337561,0.989155,-0.572553,0
BDX,CryptoNight,PoW,9.802226e+08,1400222610,Beldex,0.327547,-2.258238,0.463220,1


### Visualizing Results

In [132]:
fig = px.scatter_3d(clustered_df,
                   x='PC3',
                   y='PC2',
                   z='PC1',
                   color='class',
                   hover_name='CoinName',
                   hover_data=['Algorithm'],
                   width=600)
fig.show()

#### Scatter Plot with Tradable Cryptocurrencies

In [133]:
scaler = MinMaxScaler()

rescale_columns = ['TotalCoinsMined', 'TotalCoinSupply']
x = clustered_df[rescale_columns].values
x_scaled = scaler.fit_transform(x)

scaled_df = pd.DataFrame(x_scaled, columns=rescale_columns, index=crypto_df.index)
scaled_df["class"] = clustered_df["class"]
scaled_df["CoinName"] = clustered_df["CoinName"]

In [134]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
plot = scaled_df.hvplot(
    kind="scatter",
    x="TotalCoinsMined",
    y="TotalCoinSupply",
    c='class', 
    colormap="viridis",
    hover_cols=["CoinName"]
)

plot

:Scatter   [TotalCoinsMined]   (TotalCoinSupply,class,CoinName)

#### Table of Tradable Cryptocurrencies

In [135]:
# Table with tradable cryptos
df_columns = ['CoinName', 'Algorithm', 'ProofType', 'TotalCoinSupply', 'TotalCoinsMined', 'class']
clustered_df.hvplot.table(df_columns)

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,class]

In [137]:
# Print the total number of tradable cryptocurrencies
print(f'Total number of tradable cryptocurrencies is: {len(clustered_df)}')

Total number of tradable cryptocurrencies is: 533
